In [1]:
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr
import numpy as np

from datetime import date
from plotly import graph_objs as go
from sklearn.preprocessing import MinMaxScaler

In [2]:
START = "2015-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

scaler = MinMaxScaler(feature_range=(0,1))

In [3]:
df = pdr.DataReader('AAPL', 'yahoo', START, TODAY)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-12-31,28.282499,27.552500,28.205000,27.594999,165613600.0,25.020117
2015-01-02,27.860001,26.837500,27.847500,27.332500,212818400.0,24.782108
2015-01-05,27.162500,26.352501,27.072500,26.562500,257142000.0,24.083961
2015-01-06,26.857500,26.157499,26.635000,26.565001,263188400.0,24.086226
2015-01-07,27.049999,26.674999,26.799999,26.937500,160423600.0,24.423971


In [4]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-10-12,143.250000,141.039993,143.229996,141.509995,73035900.0,141.509995
2021-10-13,141.399994,139.199997,141.240005,140.910004,78762700.0,140.910004
2021-10-14,143.880005,141.509995,142.110001,143.759995,69907100.0,143.759995
2021-10-15,144.899994,143.509995,143.770004,144.839996,67885200.0,144.839996
2021-10-18,145.399994,143.160004,143.445007,145.304993,29626165.0,145.304993


In [9]:
df = df.reset_index()
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2014-12-31,28.282499,27.552500,28.205000,27.594999,165613600.0,25.020117
1,2015-01-02,27.860001,26.837500,27.847500,27.332500,212818400.0,24.782108
2,2015-01-05,27.162500,26.352501,27.072500,26.562500,257142000.0,24.083961
3,2015-01-06,26.857500,26.157499,26.635000,26.565001,263188400.0,24.086226
4,2015-01-07,27.049999,26.674999,26.799999,26.937500,160423600.0,24.423971


In [10]:
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()

,High,Low,Open,Close,Volume
0,28.282499,27.552500,28.205000,27.594999,165613600.0
1,27.860001,26.837500,27.847500,27.332500,212818400.0
2,27.162500,26.352501,27.072500,26.562500,257142000.0
3,26.857500,26.157499,26.635000,26.565001,263188400.0
4,27.049999,26.674999,26.799999,26.937500,160423600.0


In [11]:
data_training = pd.DataFrame(df['Close'][0: int(len(df)*0.70)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.70): int(len(df))])

data_training_array = scaler.fit_transform(data_training)

In [17]:
x_train = []
y_train = []

for i in range(100, data_training_array.shape[0]):
	x_train.append(data_training_array[i-100: i])
	y_train.append(data_training_array[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

x_train.shape

(1098, 100, 1)

In [13]:
data_training_array = scaler.fit_transform(data_training)
data_training_array

array([[0.14139561],
       [0.13398719],
       [0.11225571],
       ...,
       [0.94722352],
       [0.90750018],
       [0.92062373]])

In [14]:
from keras.layers import Dense, Dropout, LSTM

In [15]:
from keras.models import Sequential

In [19]:
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(units = 5))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100, 50)           10400     
_________________________________________________________________
dropout (Dropout)            (None, 100, 50)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 60)           26640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 60)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 80)           45120     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 80)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 120)               9

In [22]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
35/35 [==============================] - 21s 373ms/step - loss: 0.1072
Epoch 2/50
35/35 [==============================] - 11s 310ms/step - loss: 0.0262
Epoch 3/50
35/35 [==============================] - 9s 259ms/step - loss: 0.0179
Epoch 4/50
35/35 [==============================] - 9s 257ms/step - loss: 0.0159
Epoch 5/50
35/35 [==============================] - 9s 262ms/step - loss: 0.0147
Epoch 6/50
35/35 [==============================] - 9s 260ms/step - loss: 0.0138
Epoch 7/50
35/35 [==============================] - 9s 261ms/step - loss: 0.0118
Epoch 8/50
35/35 [==============================] - 9s 266ms/step - loss: 0.0120
Epoch 9/50
35/35 [==============================] - 10s 274ms/step - loss: 0.0106
Epoch 10/50
35/35 [==============================] - 9s 258ms/step - loss: 0.0099
Epoch 11/50
35/35 [==============================] - 9s 255ms/step - loss: 0.0099
Epoch 12/50
35/35 [==============================] - 9s 257ms/step - loss: 0.0089
Epoch 13/50
35/35 [===

In [23]:
model.save('keras_model.h5')

In [24]:
past_100_days = data_training.tail(100)
final_df = past_100_days.append(data_testing, ignore_index=True)
input_data = scaler.fit_transform(final_df)

In [26]:
x_test = []
y_test = []

for i in range(100, input_data.shape[0]):
	x_test.append(input_data[i-100: i])
	y_test.append(input_data[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test)
y_predicted = model.predict(x_test)
scaler = scaler.scale_

scale_factor = 1/scaler[0]
y_predicted = y_predicted * scale_factor
y_test = y_test * scale_factor

In [28]:
y_predicted

array([[17.491434, 18.035757, 17.810987, 17.72609 , 17.928488],
       [17.546465, 18.100155, 17.88369 , 17.782284, 17.989367],
       [17.730968, 18.288322, 18.082691, 17.964125, 18.17453 ],
       ...,
       [93.42479 , 92.59665 , 93.07743 , 93.404434, 92.81781 ],
       [93.660965, 92.83801 , 93.31179 , 93.64065 , 93.05798 ],
       [93.976776, 93.15796 , 93.62481 , 93.95581 , 93.37716 ]],
      dtype=float32)